# Project 2

# Purpose

This [November 2021 CNBC article](https://www.cnbc.com/2021/11/09/bitcoin-vs-gold-leading-gold-authorities-on-inflation-hedge-battle.html) on Bitcoin and gold as inflation and market risk hedges motivated this project.
I have two goals for this project:

1. To help you master data analysis
1. To help you evaluate articles in the popular media using your data analysis skills

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.float_format', '{:.4f}'.format)
%precision 4
%config InlineBackend.figure_format = 'retina'

In [ ]:
import yfinance as yf
import pandas_datareader as pdr
import requests_cache
session = requests_cache.CachedSession()

In [ ]:
import scipy.optimize as sco
import seaborn as sns
import statsmodels.formula.api as smf

# Tasks

In [ ]:
# Download Bitcoin data
btc = (
    yf.download(tickers='BTC-USD', progress=False)
    .assign(Date=lambda x: x.index.tz_localize(None))
    .set_index('Date')
    .rename_axis(columns='Variable')
)

# Download Gold data
gold = (
    yf.download(tickers='GLD', progress=False)
    .assign(Date=lambda x: x.index.tz_localize(None))
    .set_index('Date')
    .rename_axis(columns='Variable')
)

# Download PCEPI data from FRED
pcepi = pdr.DataReader(
    'PCEPI',
    start='1900',
    data_source='fred'
)

# Download SP100 data
wiki = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
wiki_ticker=(wiki[2]['Symbol']).apply(lambda x: x.replace('.', '-')).tolist()
wiki_ticker
SP100 = (
    yf.download(tickers=wiki_ticker, progress=False)
    .assign(Date=lambda x: x.index.tz_localize(None))
    .set_index('Date')
    .rename_axis(columns=['Variable', 'Ticker'])
    
)

# Download French Fama data 
ff = (
    pdr.DataReader(
        name='F-F_Research_Data_Factors_daily',
        data_source='famafrench',
        start='1900',
        session=session
    )
    [0]
    .assign(Mkt = lambda x: x['Mkt-RF'] + x['RF'])
    .div(100)
)

def port_vol(x, r, ppy):
    return np.sqrt(ppy) * r.dot(x).std()

def port_mean(x, r, ppy):
    return ppy * r.dot(x).mean()

In [ ]:
ff

In [ ]:
gold

In [ ]:
pcepi

In [ ]:
SP100

In [ ]:
btc

## Task 1: Do Bitcoin and gold hedge inflation risk?

Use the typical finance definition of [hedge](https://www.investopedia.com/terms/h/hedge.asp):

> To hedge, in finance, is to take an offsetting position in an asset or investment that reduces the price risk of an existing position. A hedge is therefore a trade that is made with the purpose of reducing the risk of adverse price movements in another asset. Normally, a hedge consists of taking the opposite position in a related security or in a derivative security based on the asset to be hedged. 

Here are a few suggestions:

1. Measure Bitcoin's price with [BTC-USD](https://finance.yahoo.com/quote/BTC-USD?p=BTC-USD&.tsrc=fin-srch) and gold's price with [GLD](https://finance.yahoo.com/quote/GLD?p=GLD&.tsrc=fin-srch)
1. Throughout the project, assume Bitcoin and U.S. public equity markets have the same closing time
1. Measure the price level with [PCEPI](https://fred.stlouisfed.org/series/PCEPI/) from the Federal Reserve Database (FRED), which is downloadable with `pdr.DataReader()`
1. Measure inflation (i.e., the rate of change in the price level) as the percent change in PCEPI

In [ ]:
# Filter data based on start_date and end_date
start_date = '2010-01-01'
end_date = '2022-04-07'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

btc = btc.reindex(gold.index).dropna()
gold = gold.reindex(btc.index).dropna()
inflation_data = inflation_data.reindex(btc.index).dropna()

In [ ]:
# Calculate price and inflation rate
btc_price = btc['Adj Close'].resample('MS').last()
gold_price = gold['Adj Close'].resample('MS').last()
inflation = pcepi['PCEPI']

In [ ]:
# # Merge data into a single data frame
# data = pd.merge(btc_returns, gold_returns, left_index=True, right_index=True)
# data = pd.merge(data, ff['RF'], left_index=True, right_index=True)
# data = pd.merge(data, ff['Mkt-RF'], left_index=True, right_index=True)
# data.columns = ['BTC', 'Gold', 'RF', 'RM']
# data

In [ ]:
# # Calculate excess returns, volatility, and expected return
# btc_excess_returns = btc_returns - inflation_data
# gold_excess_returns = gold_returns - inflation_data

# btc_volatility = btc_excess_returns.std() * np.sqrt(252)
# gold_volatility = gold_excess_returns.std() * np.sqrt(252)

# btc_expected_return = btc_excess_returns.mean() * 252 + inflation_data.mean() * 252
# gold_expected_return = gold_excess_returns.mean() * 252 + inflation_data.mean() * 252

btc_inflation_corr = btc_price.corr(inflation)
gold_inflation_corr = gold_price.corr(inflation)

In [ ]:
# # Print results
# print('Bitcoin expected return: {:.2f}%'.format(btc_expected_return * 100))
# print('Bitcoin volatility: {:.2f}%'.format(btc_volatility * 100))

# print('Gold expected return: {:.2f}%'.format(gold_expected_return * 100))
# print('Gold volatility: {:.2f}%'.format(gold_volatility * 100))

print('Correlation between Bitcoin and Inflation: {:.7f}'.format(btc_inflation_corr))
print('Correlation between Gold and Inflation: {:.7f}'.format(gold_inflation_corr))

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.plot(btc_price.index, btc_price, label='Bitcoin')
plt.plot(gold_price.index, gold_price, label='Gold')
plt.plot(inflation.index, inflation, label='Inflation Rate')
plt.xlabel('Date')
plt.ylabel('Percentage')
plt.legend()
plt.title('Bitcoin and Gold Returns vs Inflation Rate')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(btc.index, btc['Adj Close'], label='Bitcoin')
plt.plot(gold.index, gold['Adj Close'], label='Gold')
plt.plot(inflation.index, inflation, label='Inflation')
plt.xlabel('Date')
plt.ylabel('Price Level or Percent Change')
plt.legend()
plt.title('Bitcoin, Gold, and Inflation over Time')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
bar_width = 0.35
opacity = 0.8
index = np.arange(2)
plt.bar(index, [btc_expected_return, gold_expected_return], bar_width, alpha=opacity, color='b', label='Expected Return')
plt.bar(index + bar_width, [btc_volatility, gold_volatility], bar_width, alpha=opacity, color='g', label='Volatility')
plt.xlabel('Asset')
plt.ylabel('Percentage')
plt.xticks(index + bar_width/2, ('Bitcoin', 'Gold'))
plt.legend()
plt.title('Expected Return and Volatility')
plt.show()

In [ ]:
# Create a scatter plot of daily returns of Bitcoin and Gold against each other

for i in range(len(data)):
    if data['BTC'][i] > data['Gold'][i]:
        plt.scatter(data['Gold'][i], data['BTC'][i], color='blue', alpha=0.5)
    else:
        plt.scatter(data['Gold'][i], data['BTC'][i], color='orange', alpha=0.5)
plt.xlabel('Gold Returns')
plt.ylabel('Bitcoin Returns')
plt.title('Bitcoin Returns vs. Gold Returns')
plt.show()

In [ ]:
# Create scatter plot for Bitcoin returns vs. inflation rate
sns.scatterplot(
    x=btc_returns, 
    y=inflation_data, 
    alpha=0.5, 
    label='Bitcoin Returns'
)

sns.lineplot(
    x=btc_returns, 
    y=inflation_data, 
    label='Inflation Rate'
)
plt.xlabel('Date'); plt.ylabel('Percentage'); plt.title('Bitcoin Returns vs. Inflation Rate'); plt.legend();plt.show()

# Create scatter plot for Gold returns vs. inflation rate
sns.scatterplot(
    x=gold_returns, 
    y=inflation_data, 
    alpha=0.5, 
    label='Gold Returns'
)
sns.lineplot(
    x=gold_returns, 
    y=inflation_data, 
    label='Inflation Rate'
)
plt.xlabel('Date'); plt.ylabel('Percentage'); plt.title('Gold Returns vs. Inflation Rate'); plt.legend(); plt.show()

Task 1 


Based on the data above, the conclusion is that both Bitcoin and gold do not appear to effectively hedge against inflation risk.

The expected return for Bitcoin is 79.11% and for gold it is 70.58%, indicating that both assets have shown positive returns historically. However, the correlation between Bitcoin and inflation is -0.01, and the correlation between gold and inflation is -0.04, which suggests a weak or negligible relationship between the two assets and inflation. But, correlation does not necessarily imply causation, and hedge against inflation risk is a complex concept that depends on various factors beyond just correlation. It is important to note that correlation alone does not provide conclusive evidence of hedging against inflation risk.

Furthermore, the volatility (or risk) of Bitcoin is 64.99% and for gold it is 15.56%, indicating that Bitcoin is significantly more volatile compared to gold. The plot of Bitcoin and gold returns compared to the inflation rate also does not show a clear pattern of these assets effectively hedging against inflation risk, as their returns do not consistently move in opposite directions to inflation.

Therefore, based on the data and analysis, it can be concluded that Bitcoin and gold may not be reliable hedges against inflation risk. Investors should consider other assets or strategies for effectively hedging against inflation.

To better assess whether Bitcoin and gold hedge inflation risk, additional analysis and considerations are needed, such as examining the historical price trends, volatility, and performance during inflationary periods, as well as considering other factors such as market dynamics, economic conditions, and investment objectives. It may also be prudent to consult with a financial professional or conduct a comprehensive analysis using additional financial and economic data to make informed investment decisions.

## Task 2: Do Bitcoin and gold hedge market risk?

Here are a few suggestions:

1. Estimate capital asset pricing model (CAPM) regressions for Bitcoin and gold
1. Use the daily factor data from Ken French

In [ ]:
# Calculate returns and inflation rate
btc_returns = btc['Adj Close'].pct_change().dropna()
gold_returns = gold['Adj Close'].pct_change().dropna()

In [ ]:
# Merge data into a single data frame
data = pd.merge(btc_returns, gold_returns, left_index=True, right_index=True)
data = pd.merge(data, ff['RF'], left_index=True, right_index=True)
data = pd.merge(data, ff['Mkt-RF'], left_index=True, right_index=True)
data.columns = ['BTC', 'Gold', 'RF', 'RP']

In [ ]:
ff_factors = ff[['Mkt-RF', 'RF']]

In [ ]:
# Define the formula for the regression
formula = 'BTC - R ~ RP - RF'
formula_1 = 'Gold - RF ~ RP - RF'

In [ ]:
# Estimate beta coefficients for Bitcoin using OLS & print regression results for Bitcoin
btc_model = smf.ols(formula=formula, data=data).fit()
beta_btc = btc_model.params['RP']
btc_model.summary()

In [ ]:
# Estimate beta coefficients for Gold using OLS & print regression results for Gold
gold_model = smf.ols(formula=formula_1, data=data).fit()
beta_gold = gold_model.params['RP']
gold_model.summary()

In [ ]:
rf = ff_factors['RF'].mean()
market_return = ff_factors['Mkt-RF'].mean()

# Estimate expected return for Bitcoin using CAPM equation
expected_return_btc = rf + beta_btc * (market_return - rf)
print('Expected return for Bitcoin: {:.7%}'.format(expected_return_btc))

# Estimate expected return for gold using CAPM equation
expected_return_gold = rf + beta_gold * (market_return - rf)
print('Expected return for Gold: {:.7%}'.format(expected_return_gold))

In [ ]:
# Create a scatter plot of BTC returns vs. market returns
plt.scatter(data['RM'], data['BTC'])
plt.xlabel('Market Returns')
plt.ylabel('Bitcoin Returns')
plt.title('Scatter Plot of Bitcoin Returns vs. Market Returns')

In [ ]:
# Create a scatter plot of gold returns vs. market returns
plt.scatter(data['RM'], data['Gold'])
plt.xlabel('Market Returns')
plt.ylabel('Gold Returns')
plt.title('Scatter Plot of Gold Returns vs. Market Returns')
plt.show()

In [ ]:
# Create a regression plot of BTC returns vs. market returns
sns.regplot(x='RM', y='BTC', data=data)
plt.xlabel('Market Returns')
plt.ylabel('Bitcoin Returns')
plt.title('Regression Plot of Bitcoin Returns vs. Market Returns')
plt.show()

In [ ]:
# Create a regression plot of gold returns vs. market returns
sns.regplot(x='RM', y='Gold', data=data)
plt.xlabel('Market Returns')
plt.ylabel('Gold Returns')
plt.title('Regression Plot of Gold Returns vs. Market Returns')
plt.show()

In [ ]:
# Create a bar plot of expected returns for Bitcoin and gold
labels = ['Bitcoin', 'Gold']
values = [expected_return_btc, expected_return_gold]
plt.bar(labels, values)
plt.ylabel('Expected Return')
plt.title('Expected Returns for Bitcoin and Gold')
plt.show()

In [ ]:
# Create a scatter plot of daily returns of Bitcoin and gold against the market return
plt.scatter(data['RM'], data['BTC'], label='Bitcoin')
plt.scatter(data['RM'], data['Gold'], label='Gold')
plt.xlabel('Market Return')
plt.ylabel('Daily Return')
plt.legend()
plt.show()

Task 2

The possibility of financial losses brought on by the turbulence of the financial markets is referred to as market risk. This risk pertains to the possibility that an investor's portfolio value would decrease as a result of alterations in market circumstances, such as changes in stock prices, interest rates, exchange rates, and commodity prices.

Bitcoin and gold can both be used as a hedge against market risk, but they both have unique features and behavior patterns that depend on the market environment. Overall, both Bitcoin and gold can be used as hedging tools against market risk, but prior to making an investment decision, traders should think about their personal investing goals, risk appetite, and time horizon. It's also critical to keep in mind that neither asset is ensured to offer defense against all kinds of market hazards, and investors need diversify their portfolios to successfully manage risk.

Based on the above code and outputs, the beta coefficient for Bitcoin in the CAPM regression is 0.0095, indicating that Bitcoin's returns are positively correlated with the market, but at a lower magnitude than the market as a whole. This suggests that Bitcoin can provide some degree of diversification and act as a hedge against market risk.

The beta coefficient for gold in the CAPM regression is 0.0007, indicating a weaker positive correlation between gold's returns and the market compared to Bitcoin. However, it still suggests that gold can provide some degree of diversification and act as a hedge against market risk.

Furthermore, the expected return estimates for both Bitcoin and gold using the CAPM equation are positive, which suggests that they are expected to generate returns above the risk-free rate, providing some potential upside while also acting as a hedge against market risk.

In conclusion, the above analysis suggests that both Bitcoin and gold can act as hedges against market risk, with Bitcoin offering a slightly stronger correlation with the market than gold. However, it is important to note that this analysis is based on historical data and the future behavior of these assets may differ from the past. Additionally, there may be other factors that influence the behavior of these assets, such as regulatory changes, technological advancements, and investor sentiment, which could impact their effectiveness as hedges against market risk.

## Task 3: Plot the mean-variance efficient frontier of Standard & Poor's 100 Index (SP100) stocks, with and without Bitcoin and gold

Here are a few suggestions:

1. You can learn about the SP100 stocks [here](https://en.wikipedia.org/wiki/S%26P_100)
1. Only consider days with complete data for Bitcoin and gold
1. Drop any stocks with shorter return histories than Bitcoin and gold
1. Assume long-only portfolios

In [ ]:
returns_2 = SP100['Adj Close'].pct_change().loc['2004':'2022']

In [ ]:
# Renaming the columns of BTC & Gold 
series = pd.Series(data=btc_returns, name='BTC')
series1 = pd.Series(data= gold_returns, name='GOLD')

# Joining SP100, Bitcoin & Gold 
merge_df = pd.concat([returns_2, series, series1], axis=1)

# Drop any columns with fewer non-NA values than BTC or GOLD
min_non_na = merge_df[['BTC', 'GOLD']].count().min()
returns_df = merge_df.dropna(thresh=min_non_na, axis='columns')

# Only consider days with complete data for Bitcoin and gold
returns_df = returns_df.dropna()

# Drop the columns for Bitcoin and Gold
returns_df1 = returns_df.drop(['BTC', 'GOLD'], axis=1)

## Mean-variance efficient frontier of Standard & Poor's 100 Index (SP100) stocks without Bitcoin and gold

In [ ]:
tret = 252 * np.linspace(returns_df1.mean().min(), returns_df1.mean().max(), 25)
tret

In [ ]:
res_ef = []

for t in tret:
    _ = sco.minimize(
        fun=port_vol, # minimize portfolio volatility
        x0=np.ones(returns_df1.shape[1]) / returns_df1.shape[1], # initial portfolio weights
        args=(returns_df1, 252), # additional arguments to fun, in order
        bounds=[(0, 1) for c in returns_df1.columns], # bounds limit the search space for each portfolio weight
        constraints=(
            {'type': 'eq', 'fun': lambda x: x.sum() - 1}, # constrain sum of weights to one
            {'type': 'eq', 'fun': lambda x: port_mean(x=x, r=returns_df1, ppy=252) - t} # constrains portfolio mean return to the target return

        )
    )
    res_ef.append(_)

In [ ]:
for r in res_ef:
    assert r['success'] 

In [ ]:
ef = pd.DataFrame(
    {
        'tret': tret,
        'tvol': np.array([r['fun'] if r['success'] else np.nan for r in res_ef])
    }
)

ef.head()

In [ ]:
ef.mul(100).plot(x='tvol', y='tret', legend=False)
plt.ylabel('Annualized Mean Return (%)')
plt.xlabel('Annualized Volatility (%)')
plt.title(
    f'Efficient Frontier'  +
    f'\nfrom {returns_df1.index[0]:%B %d, %Y} to {returns_df1.index[-1]:%B %d, %Y}'
)

for t, x, y in zip(
    returns_df1.columns, 
    returns_df1.std().mul(100*np.sqrt(252)),
    returns_df1.mean().mul(100*252)
):
    plt.annotate(text=t, xy=(x, y))
    
plt.show()

## Mean-variance efficient frontier of Standard & Poor's 100 Index (SP100) stocks with Bitcoin and gold

In [ ]:
tret1 = 252 * np.linspace(returns_df.mean().min(), returns_df.mean().max(), 25)
tret1

In [ ]:
res_ef1 = []

for t in tret1:
    _ = sco.minimize(
        fun=port_vol, # minimize portfolio volatility
        x0=np.ones(returns_df.shape[1]) / returns_df.shape[1], # initial portfolio weights
        args=(returns_df, 252), # additional arguments to fun, in order
        bounds=[(0, 1) for c in returns_df.columns], # bounds limit the search space for each portfolio weight
        constraints=(
            {'type': 'eq', 'fun': lambda x: x.sum() - 1}, # constrain sum of weights to one
            {'type': 'eq', 'fun': lambda x: port_mean(x=x, r=returns_df, ppy=252) - t} # constrains portfolio mean return to the target return

        )
    )
    res_ef1.append(_)

In [ ]:
for r in res_ef1:
    assert r['success']

In [ ]:
ef = pd.DataFrame(
    {
        'tret1': tret1,
        'tvol1': np.array([r['fun'] if r['success'] else np.nan for r in res_ef1])
    }
)

ef.head()

In [ ]:
ef.mul(100).plot(x='tvol1', y='tret1', legend=False)
plt.ylabel('Annualized Mean Return (%)')
plt.xlabel('Annualized Volatility (%)')
plt.title(
    f'Efficient Frontier'  +
    f'\nfrom {returns_df.index[0]:%B %d, %Y} to {returns_df.index[-1]:%B %d, %Y}'
)

for t, x, y in zip(
    returns_df.columns, 
    returns_df.std().mul(100*np.sqrt(252)),
    returns_df.mean().mul(100*252)
):
    plt.annotate(text=t, xy=(x, y))
    
plt.show()

## Task 4: Find the maximum Sharpe Ratio portfolio of SP100 stocks, with and without Bitcoin and gold

Follow the data requirements of task 3.

In [ ]:
def port_sharpe(x, r, ppy, tgt):
    """
    x: portfolio weights
    r: data frame of returns
    ppy: periods per year for annualization
    tgt: target or benchmark
    """
    rp = r.dot(x) # portfolio return
    er = rp.sub(tgt).dropna() # portfolio excess return
    return np.sqrt(ppy) * er.mean() / er.std() # portfolio Sharpe Ratio

In [ ]:
def port_sharpe_neg(x, r, ppy, tgt):
    return -1 * port_sharpe(x, r, ppy, tgt)

## Maximum Sharpe Ratio portfolio of SP100 stocks without Bitcoin and gold

In [ ]:
res_sharpe = sco.minimize(
    fun=port_sharpe_neg,
    x0=np.ones(returns_df1.shape[1]) / returns_df1.shape[1],
    args=(returns_df1, 252, 0),
    bounds=[(0,1) for _ in range(returns_df1.shape[1])],
    constraints=(
        {'type': 'eq', 'fun': lambda x: x.sum() - 1}
    )
)

In [ ]:
port_sharpe(x=res_sharpe['x'], r=returns_df1, ppy=252, tgt=0)

In [ ]:
(res_sharpe['x'] > 0.00001).sum()

In [ ]:
weights = pd.Series(data=res_sharpe['x'], index=returns_df1.columns)
weights.nlargest(15).plot(kind='barh')

## Maximum Sharpe Ratio portfolio of SP100 stocks with Bitcoin and gold

In [ ]:
res_sharpe_1 = sco.minimize(
    fun=port_sharpe_neg,
    x0=np.ones(returns_df.shape[1]) / returns_df.shape[1],
    args=(returns_df, 252, 0),
    bounds=[(0,1) for _ in range(returns_df.shape[1])],
    constraints=(
        {'type': 'eq', 'fun': lambda x: x.sum() - 1}
    )
)

res_sharpe_1


In [ ]:
port_sharpe(x=res_sharpe_1['x'], r=returns_df, ppy=252, tgt=0)

In [ ]:
(res_sharpe_1['x'] > 0.00001).sum()

In [ ]:
weights = pd.Series(data=res_sharpe_1['x'], index=returns_df.columns)
weights.nlargest(15).plot(kind='barh')

In [ ]:
# (
#     pd.DataFrame(
#         data={
#             'L1':res_sharpe['x'], 
#             'L2':res_sharpe_1['x']
#         },
#         index=returns_2.columns
#     )
#     .rename_axis('Portfolio Weight')
#     .plot(kind='barh')
# )
# plt.title('Comparison Max. Sharpe Ratio Portfolio Weights')
# plt.show()

## Task 5: Every full calendar year, compare the $\frac{1}{n}$ portfolio with the out-of-sample performance of the previous maximum Sharpe Ratio portfolio

Follow the data requirements of task 3.
Estimate the previous maximum Sharpe Ratio portfolio using data from the previous two years.
Consider, at least, the Sharpe Ratios of each portfolio, but other performance measures may help you tell a more complete story.

In [ ]:
# start_date = '2010-01-01'
# end_date = '2023-04-17'
# pcepi = pdr.DataReader('PCEPI', 'fred', start_date, end_date)
# btc = yf.download('BTC-USD', start=start_date, end=end_date)
# gld = yf.download('GLD', start=start_date, end=end_date)
# inflation = (pcepi.iloc[-1]['PCEPI'] / pcepi.iloc[0]['PCEPI'] - 1) * 100
# btc_real_returns = (btc['Adj Close'] / btc['Adj Close'].shift(1)) / (1 + inflation / 100) - 1
# gld_real_returns = (gld['Adj Close'] / gld['Adj Close'].shift(1)) / (1 + inflation / 100) - 1
# df = pd.concat([btc_real_returns, gld_real_returns], axis=1)
# df.columns = ['BTC_Returns', 'GLD_Returns']
# df['Inflation'] = inflation
# df = df.dropna()

In [ ]:
# X = sm.add_constant(df['Inflation'])
# results_btc = sm.OLS(df['BTC_Returns'], X).fit()
# results_gld = sm.OLS(df['GLD_Returns'], X).fit()
# print("Bitcoin Regression Results:")
# print(results_btc.summary())
# print("Gold Regression Results:")
# print(results_gld.summary())

# # Fetch S&P 100 stock data using pandas-datareader
# sp100_stocks = pdr.data.get_data_yahoo(tickers='^OEX')['Adj Close'].to_frame()

# # Fetch Bitcoin and gold data using pandas-datareader
# bitcoin = pdr.data.get_data_yahoo(tickers='BTC-USD')['Adj Close']
# gold = pdr.data.get_data_yahoo(tickers='GC=F')['Adj Close']

# # Drop stocks with shorter return histories than Bitcoin and gold
# returns = sp100_stocks.pct_change().dropna()
# returns = returns.join(bitcoin.pct_change().rename('BTC-USD'))
# returns = returns.join(gold.pct_change().rename('GC=F'))
# valid_returns = returns.dropna(subset=['BTC-USD', 'GC=F'])

In [ ]:
# Define a function to find the maximum Sharpe Ratio portfolio
def max_sharpe_ratio_portfolio(returns_df1):
    # Calculate mean returns and covariance matrix
    mean_returns = returns_df1.mean()
    cov_matrix = returns_df1.cov()

    # Number of assets
    num_assets = len(mean_returns)

    # Generate random weights for portfolio simulation
    np.random.seed(42)
    num_portfolios = 100
    weights = np.random.random((num_portfolios, num_assets))
    weights /= np.sum(weights, axis=1).reshape((-1, 1))

    portfolio_risks = np.sqrt(np.diag(np.dot(weights, np.dot(cov_matrix, weights.T))))

    # Calculate portfolio returns, risks, and sharp ratios
    portfolio_returns = np.dot(weights, mean_returns)
    sharp_ratios = portfolio_returns / portfolio_risks

    # Find the portfolio with the highest sharp ratio
    optimal_portfolio_index = np.argmax(sharp_ratios)
    optimal_weights = weights[optimal_portfolio_index]

    return optimal_weights, mean_returns, cov_matrix

In [ ]:
# Calculate maximum Sharpe Ratio portfolio for the first two years
start_date = '2017-01-01'
end_date = '2018-12-31'
two_year_returns = returns_df1[start_date:end_date]
optimal_weights, mean_returns, cov_matrix = max_sharpe_ratio_portfolio(two_year_returns)

In [ ]:
# Calculate 1/n portfolio for the first year
start_date = '2019-01-01'
end_date = '2019-12-31'
one_year_returns = returns_df1[start_date:end_date]
num_assets = len(one_year_returns.columns)
weights = np.ones(num_assets) / num_assets
portfolio_returns = np.dot(one_year_returns, weights)
portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
portfolio_sharpe_ratio = portfolio_returns / portfolio_risk

In [ ]:
# # Compare 1/n portfolio with maximum Sharpe Ratio portfolio for every subsequent year
# for year in range(2020, 2023):
#     start_date = f'{year}-01-01'
#     end_date = f'{year}-12-31'
#     one_year_returns = returns_df1[start_date:end_date]
    
#     # Calculate out-of-sample performance of maximum Sharpe Ratio portfolio
#     portfolio_returns = np.dot(one_year_returns, optimal_weights)
#     portfolio_risk = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
#     portfolio_sharpe_ratio = portfolio_returns / portfolio_risk
    
#     # Calculate 1/n portfolio for the year
#     num_assets = len(one_year_returns.columns)
#     weights = np.ones(num_assets) / num_assets
#     one_n_portfolio_returns = np.dot(one_year_returns, weights)
#     one_n_portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
#     one_n_portfolio_sharpe_ratio = one_n_portfolio_returns / one_n_portfolio_risk
    
#     # Print results for each year
#     print(f'Year: {year}')
#     rounded_pr = np.around(portfolio_returns, 3)
#     rounded_risk = np.around(portfolio_risk, 3)
#     rounded_sharpe = np.around(portfolio_sharpe_ratio, 3)
#     o_n_pr = np.around(one_n_portfolio_returns, 3)
#     o_n_prisk = np.around(one_n_portfolio_risk, 3)
#     o_n_psr = np.around(one_n_portfolio_sharpe_ratio,3)
#     print(f'Maximum Sharpe Ratio Portfolio:\nReturns: {rounded_pr}, Risk: {rounded_risk}, Sharpe Ratio: {rounded_sharpe}\nWeights: {optimal_weights}\n')
#     print(f'1/n Portfolio:\nReturns: {o_n_pr}, Risk: {o_n_prisk}, Sharpe Ratio: {o_n_psr}\nWeights: {weights}\n')

## Task 6: What do you conclude about Bitcoin and gold as inflation and market risk hedges?

What are your overall conclusions and limitations of your analysis?
What do the data suggest about the article that motivated this project?
Please see the link at the top of this notebook.

Similar to gold, Bitcoin isn't tied to a particular currency or economy. It also isn't controlled by a small group of companies or stakeholders. Rather, it is an international asset class that reflects global demand. In times of high U.S. inflation, investors must take on more risk to offset the decline in existing asset values. For example, a 3% dividend yield may normally supplement income in retirement. But if inflation is 6%, it simply isn't good enough. Bitcoin could be one of the better options outside of equities because it sidesteps many of the political and economic risks associated with the U.S. stock market. In this vein, Bitcoin and other cryptocurrencies are one of the most practical and simple ways for an American to diversify away from purely American revenue, income, and assets.

Lessons about gold’s track record as an investing hedge may be learned by looking back at the 1970s, when the U.S. experienced its last bout of high inflation.Oil price shocks and energy shortages drove average annual U.S. inflation up to around 8.8% from 1973 to 1979. During those six years, gold won over many investors as a top inflation hedge, since the yellow metal generated an impressive 35% annual return.
Therefore, based on the data and analysis, it can be concluded that Bitcoin and gold may be reliable hedges against inflation risk. Investors should consider other assets or strategies for effectively hedging against inflation.

To better assess whether Bitcoin and gold hedge inflation risk, additional analysis and considerations are needed, such as examining the historical price trends, volatility, and performance during inflationary periods, as well as considering other factors such as market dynamics, economic conditions, and investment objectives. It may also be prudent to consult with a financial professional or conduct a comprehensive analysis using additional financial and economic data to make informed investment decisions.

Data Limitations: The analysis is based on historical data for Bitcoin, gold, and inflation, and may not necessarily reflect future performance. Additionally, the data used for analysis is limited to specific time periods, which may not capture all market conditions or events that could impact the results.
Market Volatility: Bitcoin and gold prices can be highly volatile, and their performance may be influenced by a wide range of factors including regulatory changes, market sentiment, technological developments, and macroeconomic conditions. These factors may not be fully captured in the analysis, and can impact the conclusions drawn.
Other Factors: The analysis does not consider other potential factors that could impact the hedging effectiveness of Bitcoin and gold, such as transaction costs, liquidity, and trading restrictions, among others.

Data quality and reliability: The accuracy and reliability of the daily factor data from Ken French, as well as the availability and quality of data for Bitcoin and gold, could impact the accuracy and robustness of the CAPM regressions. Data errors, missing data, or inconsistencies in data sources could introduce biases or errors in the results.

Assumptions of CAPM: The CAPM model assumes that the relationship between expected returns and market risk is linear and that all relevant risk factors are captured by the market factor. However, this may not hold true for Bitcoin and gold, as they are unique assets with different risk characteristics compared to traditional financial assets, and their expected returns may not be solely determined by market risk. This could affect the validity of the CAPM regression results and their implications.

Limitations of regression analysis: Regression analysis has its own limitations, such as the potential for spurious correlations, omitted variable bias, and multicollinearity. These limitations could impact the accuracy and robustness of the CAPM regression results for Bitcoin and gold.
Market dynamics: Bitcoin and gold markets are known for their high volatility and unique market dynamics. The CAPM model may not fully capture these dynamics, which could affect the accuracy and reliability of the regression results. Factors such as liquidity, trading volumes, and market sentiment may impact the risk and return profiles of Bitcoin and gold, but may not be fully captured by the CAPM model.

Assumptions of mean-variance optimization: Mean-variance optimization assumes that asset returns follow a normal distribution and that investors are risk-averse and make decisions solely based on expected returns and risks. These assumptions may not always hold in reality, and the results of mean-variance optimization may not accurately reflect the actual risk and return characteristics of the portfolio.

Long-only portfolio assumption: Assuming long-only portfolios may not reflect real-world investment strategies, as investors may employ short positions or other strategies that deviate from the long-only assumption. This could impact the results of the efficient frontier analysis and the conclusions drawn from it.

Risk and return trade-offs: The efficient frontier represents a trade-off between risk and return, and the optimal portfolio allocation depends on an investor's risk tolerance and investment objectives. The results may vary depending on individual investors' risk preferences, time horizon, and other personal factors.

Lack of consideration for transaction costs and taxes: The analysis may not consider transaction costs, taxes, and other fees associated with buying and selling assets, which can impact the actual returns of a portfolio in real-world investing.

Assumptions of Sharpe Ratio: The Sharpe Ratio assumes that asset returns follow a normal distribution, and it measures the risk-adjusted performance of a portfolio based on historical returns. However, actual returns of assets may deviate from normal distribution, and historical returns may not necessarily reflect future performance.

Assumptions of performance measures: Performance measures, such as Sharpe Ratio or other performance metrics, are based on historical returns and assumptions about risk and return. However, actual returns of assets may deviate from historical trends, and assumptions used in performance measures may not necessarily hold true in the future.

Sample size and data availability: The performance of a portfolio estimated using data from only two years may not be statistically significant, especially if the data set is small or if there are data gaps or missing data. This can impact the reliability and stability of the results.
Out-of-sample performance: Estimating the performance of the previous maximum Sharpe Ratio portfolio using data from the previous two years represents an out-of-sample analysis, and the results may not necessarily reflect future performance. Historical performance may not be indicative of future results, and past performance should not be solely relied upon for making investment decisions.

Individual investor circumstances: The performance of different portfolios, including the 1/n portfolio and the previous maximum Sharpe Ratio portfolio, may vary depending on individual investor circumstances, such as risk tolerance, investment objectives, time horizon, and other personal factors. It's important to consider individual investor circumstances and customize investment strategies accordingly.

Assumptions of performance measures: Performance measures, such as Sharpe Ratio or other performance metrics, are based on historical returns and assumptions about risk and return. However, actual returns of assets may deviate from historical trends, and assumptions used in performance measures may not necessarily hold true in the future.

Sample size and data availability: The performance of a portfolio estimated using data from only two years may not be statistically significant, especially if the data set is small or if there are data gaps or missing data. This can impact the reliability and stability of the results.
Out-of-sample performance: Estimating the performance of the previous maximum Sharpe Ratio portfolio using data from the previous two years represents an out-of-sample analysis, and the results may not necessarily reflect future performance. Historical performance may not be indicative of future results, and past performance should not be solely relied upon for making investment decisions.

Individual investor circumstances: The performance of different portfolios, including the 1/n portfolio and the previous maximum Sharpe Ratio portfolio, may vary depending on individual investor circumstances, such as risk tolerance, investment objectives, time horizon, and other personal factors. It's important to consider individual investor circumstances and customize investment strategies accordingly.


In conclusion we understand that both Bitcoin and Gold have been shown to exhibit low correlations to some extent with other stocks. Addition of Bitcoin and Gold can potentially offer diversification benefits in a portfolio and aids to low volatility. The above analysis uses CAPM, mean-variance optimization, to evaluate the risk and return profiles of bitcoin and gold, given their unique characteristics and market dynamics.

# Criteria

1. ***Discuss and explain your findings for all 6 tasks, and be specific!***
1. ***Your goal is to convince me of your calculations and conclusions***
1. All tasks are worth 16.67 points each
1. Your report should not exceed 25 pages
1. Here are more tips
    1. Each task includes suggestions
    1. I suggest you include plots and calculations for all but the last task
    1. Remove unnecessary code, outputs, and print statements
    1. Write functions for plots and calculations that you use more than once
    1. I will not penalize code style, but I will penalize submissions that are difficult to follow or do not follow these instructions
1. How to submit your project
    1. Restart your kernel, run all cells, and save your notebook
    1. Export your notebook to PDF (`File > Save And Export Notebook As ... > PDF` in JupyterLab)
        1. If this export does not work, you can either (1) Install MiKTeX on your laptop with default settings or (2) use DataCamp Workspace to export your notebook to PDF
        1. You do not need to re-run your notebook to export it because notebooks store output cells
    1. Upload your notebook and PDF to Canvas
    1. Upload your PDF only to Gradescope and tag your tasks and teammates
    1. Gradescope helps me give better feedback more quickly, but it is not reliable for sharing and storing your submission files

In [ ]:
# # Only consider days with complete data for Bitcoin and gold
# returns_df = returns_df.dropna()

# # Drop the columns for Bitcoin and Gold
# returns_df1 = returns_df.drop(['BTC', 'GOLD'], axis=1)

# r = returns_df.loc['2015':'2017']
# r1 = returns_df1.loc['2015':'2017']

In [ ]:
# r_sharpe = sco.minimize(
#     fun=port_sharpe_neg,
#     x0=np.ones(r.shape[1]) / r.shape[1],
#     args=(r, 252, 0),
#     bounds=[(0,1) for _ in range(r.shape[1])],
#     constraints=(
#         {'type': 'eq', 'fun': lambda x: x.sum() - 1}
#     )
# )


In [ ]:
# (r_sharpe['x'] > 0.00001).sum()

In [ ]:
# weights = pd.Series(data=r_sharpe['x'], index=returns_df.columns)
# weights.nlargest(15).plot(kind='barh')

In [ ]:
# # Define the number of assets in the portfolio
# n_assets = len(returns_df1.columns)

# # Define the weights for the 1/n portfolio
# weights = np.full(n_assets, 1/n_assets)

# # Compute the portfolio returns
# portfolio_returns = np.dot(returns_df1, weights)